<a href="https://colab.research.google.com/github/ev1025/happy_dog_map/blob/main/%EA%B0%95%EC%95%84%EC%A7%80%EB%B3%84%EB%B3%84%EC%A7%80%EB%8F%84_%ED%81%AC%EB%A1%A4%EB%A7%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from bs4 import BeautifulSoup
import requests
import time
import pandas as pd
import numpy as np
import csv

from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver import ActionChains

In [ ]:
# # 네이버 강아지 별별지도
# BASE_URL = "https://campaign.naver.com/save-the-stars/"

# def get_page(page_url):
#     page = requests.get(page_url) # 해당 url의 html을 가져옴
#     soup = BeautifulSoup(page.content, 'html.parser') # html을 .content로 문자열로 만들고 html.parser가 html형식(DOM)으로 인식하여 찾아줌
#     return soup, page 

# soup, page = get_page(BASE_URL)
# dogMapList = soup.select("#new li.items > a") # soup에서 선택자에 해당하는 요소 찾아오기

# def get_link(item):
#     return item.attrs["href"] # 요소의 객체를 받아옴

# def is_Seoul(item):
#     return '서울' in item.select(".sub_tit")[0].text

# # 별별 강아지도 서울의 url만 저장
# star_links = list(map(get_link, list(filter(is_Seoul, dogMapList))))
star_links = ["https://naver.me/5WBTxFeW",
              "https://naver.me/5oiJaUts",
              "https://naver.me/xR0apGFE",
              "https://naver.me/GsOuEGgP"]

# 셀레니움 브라우저
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-logging"])
browser = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)


# 크롤링 정보 저장할 리스트
crawl_list_2 = []

# 셀레니움으로 강아지 별별지도 접속
for url in star_links[3:4]:
    browser.get(url)
    browser.implicitly_wait(10)

    # iframe 이동
    try:
        browser.switch_to.default_content()
        browser.switch_to.frame("myPlaceBookmarkListIframe")
    except:
        time.sleep(2)
        browser.switch_to.default_content()
        browser.switch_to.frame("myPlaceBookmarkListIframe")

    # =============== 스크롤링 시작 =====================
    last_height = browser.execute_script("return document.body.scrollHeight")
    while True:
        # 스크롤 1회 다운
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1.5)
        
        # 스크롤 다운 후 스크롤 높이 다시 가져옴
        new_height = browser.execute_script("return document.body.scrollHeight")
        
        # 이전 height와 새로운 height와 같으면 탈출(스크롤을 내려도 갱신값이 없다는 뜻)
        if new_height == last_height:
            break
        last_height = new_height
    # =============== 스크롤링 끝 =====================

    # 스크롤 맨위로 올리기
    time.sleep(1)
    browser.find_element(By.CSS_SELECTOR, "body").send_keys(Keys.HOME)
    #__next > div:nth-child(2) > div.place_modal_content_web.afAvNc._6EdYRV > div._29rZGD > button
    # 현재 페이지의 가게이름들 객체 저장
    time.sleep(3)
    button_box = browser.find_elements(By.CSS_SELECTOR, 'span._2gfp-T')
    print(f'현재 페이지 가게 수 : {len(button_box)}')
    
    # 가게이름 하나씩 클릭
    for button in button_box:
        try:
            webdriver.ActionChains(browser).move_to_element(button).click(button).perform()

            # iframe이동(가게 상세 페이지)
            try:
                browser.switch_to.default_content()
                browser.switch_to.frame('entryIframe')
            except:
                browser.implicitly_wait(2)
                browser.find_element(By.CSS_SELECTOR, "#__next > div:nth-child(2) > div.place_modal_content_web.afAvNc._6EdYRV > div._29rZGD > button").send_keys(Keys.ENTER)
                browser.switch_to.default_content()
                browser.switch_to.frame('entryIframe')

            # 자바스크립트 형식을 문자열 형식으로 받아오기
            js_script = "document.querySelector(\"#app-root > div > div > div\").innerHTML"
            raw = browser.execute_script("return " + js_script)

            # innerHTML이 문자열 형식으로 바꿔줘서 raw.content가 아닌 raw로 지정
            html = BeautifulSoup(raw, "html.parser")

            time.sleep(0.5)

            # 데이터 저장하기
            temp = []
            name = html.select_one("span.Fc1rA").text
            category = html.select_one("span.DJJvD").text
            try: 
                vis_review = int(html.select_one('div.dAsGb > span:nth-child(1) > a > em').text)
            except:
                vis_review = 0
            try:
                blog_review = int(html.select_one('div.dAsGb > span:nth-child(2) > a > em').text)
            except:
                blog_review = 0
            review_num = vis_review + blog_review
            try:
                star = float(html.select_one('div.dAsGb > span.PXMot.LXIwF > em').text)
            except:
                star = np.NaN
            adr = html.select_one('span.LDgIH').text.split()
            address = adr[1] # 주소에서 행정구만 저장
            try:
                tel = html.select_one('span.xlx7Q').text
            except:
                tel = np.NaN
            
            # 뽑아온 정보 리스트에 저장
            temp.extend([name,category,review_num,star,address,tel])
            crawl_list_2.append(temp)
            
            # 다음 게가게 클릭을 위해 iframe 이동
            try:
                browser.switch_to.default_content()
                browser.switch_to.frame("myPlaceBookmarkListIframe")
            except:
                time.sleep(2)
                browser.switch_to.default_content()
                browser.switch_to.frame("myPlaceBookmarkListIframe")

        except:
            # 검색결과 누를 때 폐업한 가게 오류나면 확인버튼 클릭하고 다음 loop으로 넘어감
            browser.switch_to.default_content()
            browser.switch_to.frame("myPlaceBookmarkListIframe")      
            error_button = browser.find_element(By.XPATH, '//*[@id="__next"]/div[2]/div[2]/div[2]/button')
            error_button.click()
            continue

    
    print(f'{len(crawl_list_2)}개 완료')

# csv 저장
# 파일명 /'w'쓰기모드 / 한글인코딩 / 자동줄바꿈 하지않음
f = open(f'star_dog_map.csv', 'w', encoding='utf-8-sig', newline='') 
csvwriter = csv.writer(f)

# 컬럼명
csvwriter.writerow(['상호명','업종','리뷰 수','별점','행정구','전화번호'])
# 데이터 입력
csvwriter.writerows(crawl_list_2)

f.close()
browser.close()

[WDM] - Downloading: 100%|██████████| 6.79M/6.79M [00:00<00:00, 12.5MB/s]


현재 페이지 가게 수 : 655
652개 완료
